## Netherlands Rent Prediction

Given *data about properties in the Netherlands*, let's try to predict the **rent** for a given property.

We will use a random forest pipeline regression model to make our predictions.

Data source: https://www.kaggle.com/datasets/juangesino/netherlands-rent-properties

### Getting Started

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestRegressor

In [4]:
data = pd.read_json('archive/properties.json', lines=True)
data

,_id,externalId,areaRaw,areaSqm,city,coverImageUrl,crawlStatus,crawledAt,datesPublished,firstSeenAt,furnish,lastSeenAt,latitude,longitude,postalCode,postedAgo,propertyType,rawAvailability,rent,rentDetail,rentRaw,source,title,url,additionalCosts,additionalCostsRaw,deposit,depositRaw,descriptionNonTranslated,descriptionNonTranslatedRaw,descriptionTranslated,descriptionTranslatedRaw,detailsCrawledAt,energyLabel,gender,internet,isRoomActive,kitchen,living,matchAge,matchAgeBackup,matchCapacity,matchGender,matchGenderBackup,matchLanguages,matchStatus,matchStatusBackup,pageDescription,pageTitle,pets,registrationCost,registrationCostRaw,roommates,shower,smokingInside,toilet,userDisplayName,userId,userLastLoggedOn,userMemberSince,userPhotoUrl,additionalCostsDescription
0,{'$oid': '5d2b113a43cbfd7c77a998f4'},room-1686123,14 m2,14,Rotterdam,https://resources.kamernet.nl/image/913b4b03-5...,done,{'$date': '2019-07-26T22:18:23.018+0000'},"[{'$date': '2019-07-14T11:25:46.511+0000'}, {'...",{'$date': '2019-07-14T11:25:46.511+0000'},Unfurnished,{'$date': '2019-07-26T22:18:23.142+0000'},51.896601,4.514993,3074HN,4w,Room,26-06-'19 - Indefinite period,500,,"€ 500,-",kamernet,West-Varkenoordseweg,https://kamernet.nl/en/for-rent/room-rotterdam...,50.0,\n € 50\n ...,500.0,\n € 500\n ...,"Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...","Nice room for rent, accros the Feyenoord stadi...","\nNice room for rent, accros the Feyenoord sta...",{'$date': '2019-07-22T07:10:41.849+0000'},Unknown,Mixed,Yes,true,Shared,None,16 years -\n 99 years,16 years -\n 99 years,1 person,Not important,Not important,Not important,Not important,Not important,"Room for rent in Rotterdam, West-Varkenoordse...",Room for rent in Rotterdam €500 | Kamernet,No,0,\n € 0\n ...,5,Shared,No,Shared,Huize west,4680711.0,21-07-2019,26-06-2019,https://resources.kamernet.nl/Content/images/s...,NaN
1,{'$oid': '5d2b113a43cbfd7c77a9991a'},studio-1691193,30 m2,30,Amsterdam,https://resources.kamernet.nl/image/5e11d6b5-8...,done,{'$date': '2019-08-10T22:28:46.099+0000'},"[{'$date': '2019-07-14T11:25:46.677+0000'}, {'...",{'$date': '2019-07-14T11:25:46.677+0000'},Furnished,{'$date': '2019-08-10T22:28:46.229+0000'},52.370200,4.920721,1018AS,4w,Studio,15-08-'19 - Indefinite period,950,Utilities incl.,"€ 950,- Utilities incl.",kamernet,Parelstraat,https://kamernet.nl/en/for-rent/studio-amsterd...,0.0,\n € 0\n ...,895.0,\n € 895\n ...,"Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...","Efficiently furnished, with a large balcony, a...","\nEfficiently furnished, with a large balcony,...",{'$date': '2019-07-22T06:29:33.112+0000'},Unknown,Unknown,Yes,true,Own,Own,18 years -\n 99 years,18 years -\n 99 years,1 person,Not important,Not important,Not important,"Working student, Working","Working student, Working","Studio for rent in Amsterdam, Parelstraat, fo...",Studio for rent in Amsterdam €950 | Kamernet,No,0,\n € 0\n ...,None,Own,No,Own,Cor,1865530.0,20-07-2019,05-01-2012,https://resources.kamernet.nl/Content/images/p...,NaN
2,{'$oid': '5d2b113a43cbfd7c77a99931'},room-1690545,11 m2,11,Amsterdam,https://resources.kamernet.nl/image/74b93a27-a...,done,{'$date': '2019-10-02T22:00:33.141+0000'},"[{'$date': '2019-07-14T11:25:46.834+0000'}, {'...",{'$date': '2019-07-14T11:25:46.834+0000'},Furnished,{'$date': '2019-10-02T22:00:33.264+0000'},52.350880,4.854786,1075SB,09 Jul,Room,01-08-'19 - Indefinite period,1000,Utilities incl.,"€ 1000,- Utilities incl.",kamernet,Zeilstraat,https://kamernet.nl/en/for-rent/room-amsterdam...,NaN,\n -\n ...,1000.0,\n € 1000\n ...,Kamer van 11m2 vlakbij het Vondelpark. Met een...,\nKamer van 11m2 vlakbij het Vondelpark. Met e...,Kamer van 11m2 vlakbij het Vondelpark. Met een...,\nKamer van 11m2 vlakbij het Vondelpark. Met e...,{'$date': '2019-07-21T08:44:32.816+0000'},Unknown,Mixed,Yes,true,Shared,Shared,16 years -\n 93 years,16 years -\n 93 years,1 person,Not important,Not i

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46722 entries, 0 to 46721
Data columns (total 62 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   _id                          46722 non-null  object 
 1   externalId                   46722 non-null  object 
 2   areaRaw                      46722 non-null  object 
 3   areaSqm                      46722 non-null  int64  
 4   city                         46722 non-null  object 
 5   coverImageUrl                46722 non-null  object 
 6   crawlStatus                  46722 non-null  object 
 7   crawledAt                    46722 non-null  object 
 8   datesPublished               46722 non-null  object 
 9   firstSeenAt                  46722 non-null  object 
 10  furnish                      46722 non-null  object 
 11  lastSeenAt                   46722 non-null  object 
 12  latitude                     46722 non-null  float64
 13  longitude       

### Preprocessing

In [6]:
df = data.copy()

In [7]:
# Use only select features
select_features = ['areaSqm', 'city', 'furnish', 'latitude', 'longitude', 'propertyType', 'rent', 'internet', 'kitchen', 'living', 'pets', 'shower', 
                   'smokingInside', 'toilet']

df = df[select_features]

In [8]:
df

,areaSqm,city,furnish,latitude,longitude,propertyType,rent,internet,kitchen,living,pets,shower,smokingInside,toilet
0,14,Rotterdam,Unfurnished,51.896601,4.514993,Room,500,Yes,Shared,None,No,Shared,No,Shared
1,30,Amsterdam,Furnished,52.370200,4.920721,Studio,950,Yes,Own,Own,No,Own,No,Own
2,11,Amsterdam,Furnished,52.350880,4.854786,Room,1000,Yes,Shared,Shared,Yes,Shared,Yes,Shared
3,16,Assen,Unfurnished,53.013494,6.561012,Room,290,Yes,Shared,None,No,Shared,Yes,Shared
4,22,Rotterdam,Unfurnished,51.932871,4.479732,Room,475,Unknown,Own,Own,No,Shared,No,Shared
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46717,28,Rotterdam,Furnished,51.928624,4.507187,Room,800,Yes,Shared,Shared,No,Shared,No,Shared
46718,16,Harmelen,Furnished,52.086568,4.959942,Room,400,Yes,Shared,Shared,No,Shared,No,Shared
46719,30,Rotterdam,Furnished,51.928624,4.507187,Room,950,Yes,Shared,Shared,No,Shared,No,Shared
46720,35,Rotterdam,Furnished,51.928624,4.507187,Room,1050,Yes,Shared,Shared,No,Shared,No,Shared


In [9]:
df.isna().sum()

areaSqm            0
city               0
furnish            0
latitude           0
longitude          0
propertyType       0
rent               0
internet         100
kitchen          100
living           100
pets             100
shower           100
smokingInside    100
toilet           100
dtype: int64

In [24]:
df['internet'].unique()

array(['Yes', 'Unknown', 'No', nan], dtype=object)

In [23]:
df.loc[df['internet'].isna(), :]

,areaSqm,city,furnish,latitude,longitude,propertyType,rent,internet,kitchen,living,pets,shower,smokingInside,toilet
77,13,Den Haag,Furnished,52.060570,4.300775,Room,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,14,Den Haag,Furnished,52.060570,4.300775,Room,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,20,Ubbena,Furnished,53.054733,6.589102,Room,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,18,Groningen,Uncarpeted,53.232240,6.572627,Room,355,NaN,NaN,NaN,NaN,NaN,NaN,NaN
231,10,Haarlem,Furnished,52.365043,4.659657,Room,350,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43453,9,Utrecht,Unfurnished,52.063441,5.124416,Room,550,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44914,26,Rotterdam,Furnished,51.894133,4.492083,Room,585,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45434,15,Amsterdam,Furnished,52.356464,4.876734,Room,650,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45492,120,Nieuwerkerk a/d IJssel,Furnished,51.959334,4.611392,Apartment,1950,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
bad_rows = df.loc[df['internet'].isna(), :].index
bad_rows

Index([   77,    80,    91,   116,   231,   348,   484,   492,   537,   553,
         555,   570,   626,   740,   749,   840,   907,   930,   932,   999,
        1089,  1122,  1192,  1271,  1337,  1350,  1352,  1565,  1615,  1942,
        2038,  2346,  2598,  2704,  2995,  3041,  3372,  3373,  3409,  3423,
        3427,  3446,  3461,  3462,  3463,  3488,  4176,  4202,  4213,  4342,
        4347,  4398,  4443,  4482,  4514,  4552,  4583,  4648,  4869,  4925,
        4932,  4972,  5368,  5378,  5641,  6019,  6020,  6275,  6361,  6493,
        6643,  6768,  6834,  6840,  6893,  6932,  6963,  6967,  8895, 10069,
       11904, 13503, 15265, 15739, 22198, 24546, 26579, 27214, 27377, 27378,
       27823, 34340, 35411, 38851, 42950, 43453, 44914, 45434, 45492, 45520],
      dtype='int64')

In [29]:
# Drop bad rows
df = df.drop(bad_rows, axis=0).reset_index(drop=True)

In [30]:
df.isna().sum()

areaSqm          0
city             0
furnish          0
latitude         0
longitude        0
propertyType     0
rent             0
internet         0
kitchen          0
living           0
pets             0
shower           0
smokingInside    0
toilet           0
dtype: int64

In [31]:
df.describe()

,areaSqm,latitude,longitude,rent
count,46622.000000,46622.000000,46622.000000,46622.000000
mean,31.636974,52.201675,5.315371,667.974111
std,29.877284,0.517286,0.798836,416.830406
min,6.000000,50.770041,3.410016,1.000000
25%,14.000000,51.925374,4.712789,395.000000
50%,20.000000,52.162522,5.083407,550.000000
75%,40.000000,52.370420,5.896593,800.000000
max,675.000000,53.434608,7.206637,5999.000000


In [33]:
{column: df[column].unique() for column in df.select_dtypes('object').columns}

{'city': array(['Rotterdam', 'Amsterdam', 'Assen', 'Groningen', 'Zeist',
        'Maastricht', 'Callantsoog', 'Alphen aan den Rijn', 'Tilburg',
        'Enschede', 'Leeuwarden', 'Eindhoven', 'Wageningen', 'Diemen',
        'Utrecht', 'Almere', 'Alkmaar', 'Harderwijk', 'Hilversum', 'Delft',
        'Den Bosch', 'Stoutenburg', 'Leiden', 'Den Haag', 'Boxtel',
        'Badhoevedorp', 'Veenendaal', 'Amstelveen', 'Nijmegen', 'Venlo',
        'Zwolle', 'Ubbena', 'Arnhem', 'Leimuiden', 'Riel', 'Nieuwegein',
        'Haren Gn', 'Uitgeest', 'Beverwijk', 'Ede', 'Nijkerk',
        'Amersfoort', 'Loosdrecht', 'Apeldoorn', 'Vaals', 'Velp',
        'Vlaardingen', 'Montfoort', 'Heemstede', 'Breda', 'Purmerend',
        'Baarn', 'Spijkenisse', 'Deventer', 'Hengelo',
        'Capelle aan den IJssel', 'Bovenkarspel', 'Weesp', 'Harskamp',
        'Zeeland', 'Waalre', 'IJsselstein', 'Pijnacker', 'Sittard',
        'Putten', 'Vlissingen', 'Haarlem', 'Rijswijk', 'Zandvoort',
        'Zutphen', 'Heerlen', 'Br

In [35]:
# Encode improper values
df = df.replace({'': np.nan, 'Unknown': np.nan})

In [36]:
df

,areaSqm,city,furnish,latitude,longitude,propertyType,rent,internet,kitchen,living,pets,shower,smokingInside,toilet
0,14,Rotterdam,Unfurnished,51.896601,4.514993,Room,500,Yes,Shared,None,No,Shared,No,Shared
1,30,Amsterdam,Furnished,52.370200,4.920721,Studio,950,Yes,Own,Own,No,Own,No,Own
2,11,Amsterdam,Furnished,52.350880,4.854786,Room,1000,Yes,Shared,Shared,Yes,Shared,Yes,Shared
3,16,Assen,Unfurnished,53.013494,6.561012,Room,290,Yes,Shared,None,No,Shared,Yes,Shared
4,22,Rotterdam,Unfurnished,51.932871,4.479732,Room,475,NaN,Own,Own,No,Shared,No,Shared
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46617,28,Rotterdam,Furnished,51.928624,4.507187,Room,800,Yes,Shared,Shared,No,Shared,No,Shared
46618,16,Harmelen,Furnished,52.086568,4.959942,Room,400,Yes,Shared,Shared,No,Shared,No,Shared
46619,30,Rotterdam,Furnished,51.928624,4.507187,Room,950,Yes,Shared,Shared,No,Shared,No,Shared
46620,35,Rotterdam,Furnished,51.928624,4.507187,Room,1050,Yes,Shared,Shared,No,Shared,No,Shared


In [37]:
df.isna().sum()

areaSqm             0
city                0
furnish           362
latitude            0
longitude           0
propertyType        0
rent                0
internet         8146
kitchen          7628
living           8551
pets                0
shower           7631
smokingInside       0
toilet           7660
dtype: int64

In [43]:
df.isna().mean()

areaSqm          False
city             False
furnish           True
latitude         False
longitude        False
propertyType     False
rent             False
internet          True
kitchen           True
living            True
pets             False
shower            True
smokingInside    False
toilet            True
dtype: bool

In [46]:
# Fill missing values
missing_value_columns = df.loc[:, df.isna().sum() > 0].columns
missing_value_columns

Index(['furnish', 'internet', 'kitchen', 'living', 'shower', 'toilet'], dtype='object')

In [47]:
for column in missing_value_columns:
    df[column] = df[column].fillna(df[column].mode()[0])

In [48]:
df.isna().sum()

areaSqm          0
city             0
furnish          0
latitude         0
longitude        0
propertyType     0
rent             0
internet         0
kitchen          0
living           0
pets             0
shower           0
smokingInside    0
toilet           0
dtype: int64

In [49]:
{column: df[column].unique() for column in df.select_dtypes('object').columns}

{'city': array(['Rotterdam', 'Amsterdam', 'Assen', 'Groningen', 'Zeist',
        'Maastricht', 'Callantsoog', 'Alphen aan den Rijn', 'Tilburg',
        'Enschede', 'Leeuwarden', 'Eindhoven', 'Wageningen', 'Diemen',
        'Utrecht', 'Almere', 'Alkmaar', 'Harderwijk', 'Hilversum', 'Delft',
        'Den Bosch', 'Stoutenburg', 'Leiden', 'Den Haag', 'Boxtel',
        'Badhoevedorp', 'Veenendaal', 'Amstelveen', 'Nijmegen', 'Venlo',
        'Zwolle', 'Ubbena', 'Arnhem', 'Leimuiden', 'Riel', 'Nieuwegein',
        'Haren Gn', 'Uitgeest', 'Beverwijk', 'Ede', 'Nijkerk',
        'Amersfoort', 'Loosdrecht', 'Apeldoorn', 'Vaals', 'Velp',
        'Vlaardingen', 'Montfoort', 'Heemstede', 'Breda', 'Purmerend',
        'Baarn', 'Spijkenisse', 'Deventer', 'Hengelo',
        'Capelle aan den IJssel', 'Bovenkarspel', 'Weesp', 'Harskamp',
        'Zeeland', 'Waalre', 'IJsselstein', 'Pijnacker', 'Sittard',
        'Putten', 'Vlissingen', 'Haarlem', 'Rijswijk', 'Zandvoort',
        'Zutphen', 'Heerlen', 'Br

In [50]:
df

,areaSqm,city,furnish,latitude,longitude,propertyType,rent,internet,kitchen,living,pets,shower,smokingInside,toilet
0,14,Rotterdam,Unfurnished,51.896601,4.514993,Room,500,Yes,Shared,None,No,Shared,No,Shared
1,30,Amsterdam,Furnished,52.370200,4.920721,Studio,950,Yes,Own,Own,No,Own,No,Own
2,11,Amsterdam,Furnished,52.350880,4.854786,Room,1000,Yes,Shared,Shared,Yes,Shared,Yes,Shared
3,16,Assen,Unfurnished,53.013494,6.561012,Room,290,Yes,Shared,None,No,Shared,Yes,Shared
4,22,Rotterdam,Unfurnished,51.932871,4.479732,Room,475,Yes,Own,Own,No,Shared,No,Shared
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46617,28,Rotterdam,Furnished,51.928624,4.507187,Room,800,Yes,Shared,Shared,No,Shared,No,Shared
46618,16,Harmelen,Furnished,52.086568,4.959942,Room,400,Yes,Shared,Shared,No,Shared,No,Shared
46619,30,Rotterdam,Furnished,51.928624,4.507187,Room,950,Yes,Shared,Shared,No,Shared,No,Shared
46620,35,Rotterdam,Furnished,51.928624,4.507187,Room,1050,Yes,Shared,Shared,No,Shared,No,Shared


In [51]:
# Split df into X and y
y = df['rent']
X = df.drop('rent', axis=1)

In [52]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True, random_state=1)

In [53]:
X_train

,areaSqm,city,furnish,latitude,longitude,propertyType,internet,kitchen,living,pets,shower,smokingInside,toilet
29791,61,Rotterdam,Unfurnished,51.925125,4.486212,Apartment,Yes,Shared,Shared,No,Shared,No,Shared
44827,45,Rotterdam,Unfurnished,51.893369,4.517075,Apartment,Yes,Own,Own,No,Own,No,Own
37089,105,Amsterdam,Furnished,52.376979,4.839116,Apartment,Yes,Own,Own,No,Own,No,Own
13269,20,Delft,Uncarpeted,51.996010,4.352954,Room,Yes,Shared,Shared,By mutual agreement,Shared,Not important,Shared
14654,44,Rotterdam,Furnished,51.891709,4.480317,Apartment,Yes,Shared,Shared,No,Shared,No,Shared
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43723,11,Enschede,Unfurnished,52.234413,6.849043,Room,Yes,Shared,Shared,No,Shared,No,Shared
32511,21,Utrecht,Furnished,52.086425,5.125311,Room,Yes,Shared,Shared,No,Shared,No,Shared
5192,16,Groningen,Furnished,53.229623,6.524759,Room,Yes,Shared,Shared,No,Shared,No,Shared
12172,7,Utrecht,Furnished,52.102119,5.096010,Room,Yes,Shared,None,No,Shared,No,Shared


In [54]:
y_train

29791    1195
44827     950
37089    1500
13269     330
14654     845
         ... 
43723     310
32511     435
5192      325
12172     175
33003    1495
Name: rent, Length: 32635, dtype: int64

### Building Pipeline and Training

In [55]:
{column: len(X[column].unique()) for column in X.select_dtypes('object').columns}

{'city': 737,
 'furnish': 3,
 'propertyType': 5,
 'internet': 2,
 'kitchen': 3,
 'living': 3,
 'pets': 3,
 'shower': 3,
 'smokingInside': 3,
 'toilet': 3}

In [61]:
nominal_features = [
    'city',
    'furnish',
    'propertyType',
    'kitchen',
    'living',
    'pets',
    'shower',
    'smokingInside',
    'toilet'
]

binary_transformer = Pipeline(steps=[
    ('ordinal', OrdinalEncoder())    
])

nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('binary', binary_transformer, ['internet']),
    ('nominal', nominal_transformer, nominal_features)
], remainder='passthrough')

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])


In [62]:
model.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('regressor', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('binary', ...), ('nominal', ...)]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


### Results

In [64]:
y_pred = model.predict(X_test)
y_pred

array([ 437.53 ,  927.085,  341.29 , ..., 1361.45 ,  369.142,  431.87 ])

In [65]:
y_test

31410     400
34454     775
42517     345
36306     500
43045     800
         ... 
42138    1600
36266     795
7926     1250
25058     365
15087     475
Name: rent, Length: 13987, dtype: int64

In [70]:
rmse = np.sqrt(np.mean((y_test - y_pred)**2))
rmse

np.float64(156.38368183022536)

In [71]:
y_test.describe()

count    13987.000000
mean       664.911561
std        413.872062
min          1.000000
25%        390.000000
50%        550.000000
75%        800.000000
max       5000.000000
Name: rent, dtype: float64

In [76]:
r2_score = 1 - (np.sum((y_test - y_pred)**2) / np.sum((y_test - y_test.mean())**2))
r2_score

np.float64(0.8572152925659318)

In [78]:
print("RMSE: {:.2f}".format(rmse))
print("R^2 Score: {:.5f}".format(r2_score))

RMSE: 156.38
R^2 Score: 0.85722
